<a href="https://colab.research.google.com/github/JasonMullen/Data-Science-Projects/blob/main/Convolutioal_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Convolution

In [ ]:
import numpy as np

italicized text# The Convolutional Class

In [ ]:
pip install upgrade -ignore-installed tensorflow

Looking in indexes: gnore-installed, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade


In [ ]:
#Implementation of forwards and backwards propogation
import numpy as np
from scipy import signal
from keras.layers import Layer

#The creation of a 4D convolutional Layer
class Convolutional(Layer):
    def __init__(self, input_shape, kernel_size, depth): #Constructor takes in three parameters
        input_depth, input_height, input_width = input_shape #unpacking the input shape
        self.depth = depth

        #Had to slightly edit the name
        self._input_shape = input_shape


        self.input_depth = input_depth
        #Had to edit this one too because of the conflicting names
        self._output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1) #Computing the output shape
        
        #There is the depth which is the number of kernals, then there is the height and the width of the output matrix
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size) #Computes the shape of the Kernals. 
        #This specific Kernal is four dimensional, there are multiple Kernels and each is a 3D block.
        self.kernels = np.random.randn(*self.kernels_shape) #Randomly initializing each Kernal
        
        #Need to fix for some reason.
        self.biases = np.random.randn(*self._output_shape) #Randomly initializing each Kernal
        
        #Implementing Feedforward(Propogation)
                #So, given the Derivative of E with respect to the output, we want to compute the derivate of E with respect to the Kernal.

    def forward(self, _input):
        self._input = _input 
        self._output = np.copy(self.biases) 
        #TODO: Implement the forward method using the formauala provided in the powerpoint.
        #You may add or remove any variables that you wish
        for i in range(self.depth): #For loop that goes throught the output depth
            for j in range(self.input_depth): #For loop that goes throught the input depth
                self._output[i] += signal.correlate2d(self._input[j],self.kernels[i,j], "valid") #Keeps updating the output and it computs the cross correlation
        return self._output
    
    #Implementing backwards(BackPropogation)
    #Updating the Kernals and their gradients
    #So, given the Derivative of E with respect to the output, we want to compute the derivate of E with respect to the Kernal.
    def backward(self, output_gradient, learning_rate):
      #Initializing empty arrays for the kernel gradient and input gradient.
      kernels_gradient = np.zeros(self.kernels_shape)
      input_gradient = np.zeros(self.input_shape)
          
          #Computing the derivative of E with respect to k,i,j
      for i in range(self.depth):
          for j in range(self.input_depth):
              kernels_gradient[i,j] = signal.correlate2d(self._input[j], output_gradient[i], "valid") #Computing the kernal gradient
              input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i,j], "full") #Computing the input gradient
      #The Bias gradients is equal to the output_gradient
      self.kernels -= learning_rate * kernels_gradient #updating the kernals using gradient descent..
      self.biases -= learning_rate *output_gradient #updating the biases using gradient descent.
      return input_gradient 

      

# Reshaping

In [ ]:
import numpy as np
from keras.layers import Layer
#The reshape layer is needed because the output layer of the convolutional layer is a 3D block.

class Reshape(Layer): 
    def __init__(self,input_shape, output_shape): #inherts the parameters from the base layers class.
        #Taking in the shape of the input and the output.
        self._input_shape = input_shape 
        self._output_shape = output_shape
        
    def forward(self, input):
        return np.reshape(input, self._output_shape) #Reshapes the input to the output shape.
    
    def backward(self, output_gradient, learning_rate):
        return np.reshape(output_gradient, self._input_shape) #Reshapes the output to the input shape.

# Binary Cross entropy 

In [ ]:
#Binary Cross entorpy is a good choice because it compares each theorized output to the actual output which is either zero or 1.
#Then it penalizes the probabilities based on the distance from the expected value.

import numpy as np

def binary_cross_entropy(y_true, y_pred):
  return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1- y_pred))

def binary_cross_entropy_prime(y_true, y_pred):
  return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

# Dense Layer

In [ ]:
import numpy as np
from keras.layers import Layer

class Dense(Layer):
    def __init__(self, input_size, output_size):
        self._weights = np.random.rand(output_size, input_size)
        self.bias = np.random.rand(output_size, 1)
        
        
    def foward(self, input):
        self.input = input 
        return np.dot(self._weights, self.input) + self.bias
        
        
    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        input_gradient = np.dot(self._weights.T, output_gradient)
        self._weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return input_gradient 
        

# Implementing Sigmoid Activtion

In [ ]:
pip install activation

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
from keras.layers import Layer

class Activation(Layer):
    def __init__(self, activation):
                 #activation_prime):
        self.activation = activation
        #self.activation_prime = activation_prime
        
    def foward(self,input):
        self.input = input
        return self.activation(self.input)
    
    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient)
                           #self.activation_prime(self.input))
    
    

In [ ]:
import numpy as np
from keras.layers import Layer

class Sigmoid(Activation):
  def __init__(self):
    def sigmoid(x):
      return 1 / (1 + np.exp(-x))
    super().__init__(sigmoid)
        #def sigmoid_prime(x):
            #s = sigmoid(x)
           # return s * (1 - s)
    


# MNIST Code implementation

In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils


def preprocessing(x,y,limit):
  zero_index = np.where(y == 0)[0][:limit]
  one_index = np.where(y == 1)[0][:limit]
  all_indices = np.hstack((zero_index, one_index))
  all_indices = np.random.permutation(all_indices)
  x,y = x[all_indices], y[all_indices]
  x = x.reshape(len(x),1,28,28)
  x = x.astype("float32") / 255
  y = np_utils.to_categorical(y)
  y = y.reshape(len(y), 2,1)
  return x,y

  #load MNIST from server, limit to 100 images per class because the CPU hasn't been optimized.
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocessing(x_train, y_train, 100)
x_test, y_test = preprocessing(x_test, y_test, 100)

In [ ]:

#The Convolutional Neural Network
network = [
    
  Convolutional((1,28,28),3,5),
  Sigmoid(),
  Reshape((5,26,26),(5*26*26,1)),
  Dense(5*26*26,100),
  Sigmoid(),
  Dense(100,2),
  Sigmoid()

]
epochs = 10
lr = 0.1


#def predict(network, input):
 #   output = input
  #  for layer in network:
   #     output = layer.forward(output)
   # return output

def train(network, loss, loss_prime, x_train, y_train,epochs = 1000, learning_rate = 0.01, verbose = True):
    for e in range(epochs):
        error = 0 
        for x,y in zip(x_train, y_train):
            #forward 
            output = x
            for layer in network:
              output = layer.forward(output)
            #error 
            error += binary_cross_entropy(y, output)
            
          
            #backward
            grad = binary_cross_entropy_prime(y, output)
            for layer in reversed(network):
                grad = layer.backward(grad, learning_rate)
            
            #Todo: Perform back prop 

    error /= len(x_train)
    print(f"{e + 1}/{epochs}, error = {error}")
    
#TODO: Run the test data through and print out your predictions
for x,y in zip(x_test, y_test):
    output = x
    for layer in network:
      output = layer.forward(output)
      print(f"pred: {np.argmax(output)}, true: {np.argmax(y)}")

pred: 2329, true: 1


AttributeError: ignored